# **XRP Ledger 토큰 발행 실습**

이번 세션에서는 xrpl-py 라이브러리를 통해 XRP Ledger 퍼블릭 테스트넷에서 지갑을 생성하고,

간단한 트랜잭션 및 토큰을 발행, 전송하는 실습을 진행해 보도록 하겠습니다.

## **필요 라이브러리 설치**

pip install로 XRP Ledger 개발에 필요한 라이브러리들을 설치합니다.

In [ ]:
!pip install xrpl-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.8/213.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.1/43.1 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00


In [ ]:
# 일반적인 환경에서는 필요 없는 부분입니다.
# colab의 특성상 async 함수 동작 시에 이 부분 처리가 필요합니다.
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

## **지갑 생성 (Wallet Creation)**

테스트넷에서 지갑을 생성하고, XRP Ledger와 통신하기 위한 클라이언트 객체를 생성합니다.

그리고 `faucet` 서비스를 통해 테스트용 xrp를 받는 실습을 진행합니다.

cryptographic-keys.svg

### **XRP Ledger 계정 생성 원리**

XRP Ledger의 비대칭 키 생성은 Elliptic Curve Cryptography(ECC) 기반입니다. 이는 매우 큰 두 개의 소수의 곱을 인수분해하는 것이 사실상 불가능하기 때문에 강력한 보안을 제공합니다.

#### **키 생성 과정**

1. **시드 생성**

    사용자는 임의의 시드를 선택하거나 생성합니다. XRP Ledger에서는 암호학적으로 안전한 난수 생성기를 사용해 16바이트의 임의의 데이터를 생성해서 원시 시드로 사용합니다. 원시 시드는 Base58Check 인코딩을 통해 다시 s로 시작하는 29자리의 문자열로 변환됩니다.

2. **비공개 키 생성**

    앞에서 생성한 원시 시드를 사용하여 타원곡선 DSA (Elliptic Curve Digital Signature Algorithm, ECDSA)를 통해 비공개 키를 생성합니다. 비공개 키를 생성하는 과정은 타원곡선의 기본 점(G)을 원시 시드(s)번 곱하는 것으로, 이 연산을 통해 타원곡선 위의 새로운 점(P)이 얻어지게 됩니다. 즉, 비공개 키는 기본적으로 타원곡선 위의 한 점입니다.

3. **공개 키 생성**

    비공개 키를 사용하여 공개 키를 생성합니다. 공개 키는 비공개 키(타원곡선 위의 점)에 타원곡선의 기본 점을 곱하여 얻어집니다. 이렇게 생성된 공개 키는 비밀 키를 알지 못하는 다른 사람들이 메시지를 암호화하거나 디지털 서명을 검증하는 데 사용될 수 있습니다.

4. **계정 ID 생성**

    공개 키는 다시 계정 ID로 해시됩니다. 이 계정 ID는 공개 키의 SHA-256 해시이며, 이 해시의 첫 20바이트만 사용됩니다.

5. **XRP Ledger 주소 생성**

    계정 ID는 최종적으로 XRP Ledger 주소로 변환됩니다. 이 변환은 Base58Check 인코딩을 사용하며, 이는 주소의 첫 글자가 'r'로 시작하게 만듭니다. 이 인코딩 방식은 오류 감지 기능을 내장하고 있어, 주소를 잘못 입력했을 때 이를 감지할 수 있게 합니다.

이러한 과정을 통해 XRP Ledger에서 사용할 수 있는 지갑이 생성됩니다. 하지만 실제 지갑 생성 시에는 직접 이 과정을 따라할 필요는 없고, 모든 과정이 자동으로 사용자의 기기에서 안전하게 이루어지게 됩니다.

In [ ]:
from xrpl.wallet import Wallet, generate_faucet_wallet
from xrpl.clients import JsonRpcClient

In [ ]:
TESTNET_URL = "https://s.altnet.rippletest.net:51234"
client = JsonRpcClient(TESTNET_URL)

In [ ]:
wallet = generate_faucet_wallet(client=client, debug=True)

Attempting to fund address rUZqTpdnyYnkeHEJh9NeDMCtwVVLmTcWqb
Faucet fund successful.


In [ ]:
wallet.__dict__

{'seed': 'sEd7tisqNGMTigf2Fn8NRxa1dpHhTsN',
 'algorithm': <CryptoAlgorithm.ED25519: 'ed25519'>,
 'public_key': 'EDB1D941F4A2B0E74011A9EB7D7C0BE1FC7F17B9A2BD77598900DD4F5E5CA0AD80',
 'private_key': 'ED290F27733E021071085175B1D3A10EE184EF36A5BFB4D93B67DFEDE14597F0CD',
 '_address': 'rUZqTpdnyYnkeHEJh9NeDMCtwVVLmTcWqb'}

In [ ]:
wallet.address

'rUZqTpdnyYnkeHEJh9NeDMCtwVVLmTcWqb'

In [ ]:
dest_wallet = generate_faucet_wallet(client)

## **트랜잭션 (Transaction)**

트랜잭션의 lifecycle에 대해 알아보고,

가장 기본적인 트랜잭션인 XRP를 보내는 Payment 트랜잭션을 실행해 보겠습니다.

In [ ]:
from xrpl.transaction import (
    autofill_and_sign,
    submit_and_wait,
    XRPLReliableSubmissionException,
)
from xrpl.models.transactions import Transaction, Payment

### **트랜잭션 Lifecycle**

트랜잭션의 생명주기는 트랜잭션 생성부터 최종적으로 레저에 포함될 때까지의 과정을 포함합니다. 각 단계는 다음과 같습니다:

1. **트랜잭션 생성**:

    트랜잭션 객체는 필요한 필드(트랜잭션 유형, 계정, 수신자, 금액 등)를 포함하여 생성됩니다. 이 단계에서는 필요한 모든 데이터를 수집하고 유효성을 검사합니다.
    [`xrpl.models.transactions`](https://xrpl-py.readthedocs.io/en/latest/source/xrpl.models.transactions.html#module-xrpl.models.transactions)에서 올바른 트랜잭션 유형을 선택하고, 필요한 필드를 채워넣어 트랜잭션을 생성할 수 있습니다.  
    필요한 필드는 직접 채우거나 [`xrpl.transaction.autofill`](https://xrpl-py.readthedocs.io/en/latest/source/xrpl.transaction.html#xrpl.transaction.autofill) 함수를 사용하여 자동으로 채워넣을 수 있습니다.

2. **트랜잭션 서명**:

    트랜잭션은 송신자의 비밀키로 서명되어야 합니다. 이는 트랜잭션의 발송인이 자신의 계정을 제어하고 트랜잭션을 생성했음을 증명합니다.

    트랜잭션 객체의 [`sign`](https://xrpl-py.readthedocs.io/en/latest/source/xrpl.transaction.html#xrpl.transaction.sign) 메서드를 사용하여 트랜잭션을 서명할 수 있습니다.

3. **트랜잭션 제출**:

    서명된 트랜잭션은 XRP Ledger에 제출되어 다음 유효성 검사 단계로 넘어갑니다. 제출된 트랜잭션은 대기열에 들어가며, 다음 레저에서 처리될 차례를 기다립니다.

    트랜잭션 객체의 [`submit`](https://xrpl-py.readthedocs.io/en/latest/source/xrpl.transaction.html#xrpl.transaction.submit) 메서드를 사용하여 트랜잭션을 제출할 수 있습니다.

4. **트랜잭션 검증**:

    레저의 새 버전을 만드는 검증자들은 제출된 트랜잭션을 검증하고 처리합니다. 트랜잭션이 유효하다면 (예: 충분한 잔고, 올바른 서명 등), 이는 새 레저에 포함됩니다.

    _트랜잭션이 레저에 포함되면, 이는 더 이상 변경될 수 없습니다._

5. **최종 트랜잭션**:

    트랜잭션이 레저에 포함되면, 이는 완료된 상태로 간주되며, 관련된 계정 상태(예: 잔고)는 이 트랜잭션에 따라 업데이트됩니다.

In [ ]:
def submit_transaction(
        client: JsonRpcClient,
        wallet: Wallet,
        transaction: Transaction,
        check_fee: bool = True,
    ) -> dict:
        """
        트랜잭션을 제출하고 그 결과를 반환합니다.

        Args:
            client (JsonRpcClient): XRPL과 통신하기 위한 클라이언트 객체 입니다.
            wallet (Wallet): 트랜잭션을 제출하는 계정의 지갑 객체입니다.
            transaction (Transaction): 제출할 트랜잭션입니다.
            check_fee (bool, optional): 수수료를 확인할지 여부입니다. 기본값으로 True를 사용합니다.

        Returns:
            dict: 트랜잭션의 결과입니다.

        Raises:
            XRPLReliableSubmissionException: 트랜잭션 제출이 실패하면 발생합니다.
        """
        # Autofill and sign transaction
        signed_tx = autofill_and_sign(
            transaction=transaction,
            client=client,
            wallet=wallet,
            check_fee=check_fee,
        )

        # Validate transaction
        signed_tx.validate()

        # Send transaction and get response
        response = submit_and_wait(
            transaction=signed_tx, client=client, wallet=wallet
        )

        # Raise exception if transaction failed
        if not response.is_successful():
            raise XRPLReliableSubmissionException(response.result)

        # Return result
        return response.result

### **Payment 트랜잭션 생성 및 제출**

XRP를 한 계정으로부터 다른 계정으로 보내는 트랜잭션을 생성하고 보내는 과정을 설명하겠습니다.

트랜잭션에 필수적인 필드들 이외에도 선택적으로 전달할 수 있는 매개변수들이 있습니다.

[XRP Ledger 공식문서](https://xrpl.org/transaction-common-fields.html)를 참고하세요.

In [ ]:
def send_xrp(
    client: JsonRpcClient,
    wallet: Wallet,
    destination_address: str,
    amount: str | int,
    **kwargs,
) -> dict:
    """
    이 계정에서 목적지 주소로 토큰을 보냅니다.

    Args:
        client (JsonRpcClient): XRPL과 통신하기 위한 클라이언트 객체 입니다.
        wallet (Wallet): 토큰을 보내는 계정의 지갑 객체입니다.
        destination_address (str): 토큰을 받을 계정의 주소입니다.
        amount (str | int): 보낼 토큰의 양입니다.
        **kwargs: 트랜잭션 설정에 추가로 전달할 선택적 매개변수입니다.

    Returns:
        dict: 트랜잭션의 결과입니다.
    """
    # create payment transaction
    payment_tx = Payment(
        account=wallet.address,
        amount=str(amount),
        destination=destination_address,
        **kwargs,
    )

    # Submit transaction and return result
    return submit_transaction(
        client=client, wallet=wallet, transaction=payment_tx, check_fee=True
    )

In [ ]:
result = send_xrp(
    client=client,
    wallet=wallet,
    destination_address=dest_wallet.address,
    amount="1000",
)

In [ ]:
result

{'Account': 'rGkJhixWWpXHPD4ebcrZAAr37bnMMguhj',
 'Amount': '1000',
 'Destination': 'rBALEkaidLzjynofASY8cfD85a5Vsp76ZQ',
 'Fee': '10',
 'Flags': 0,
 'LastLedgerSequence': 39806425,
 'Sequence': 39806402,
 'SigningPubKey': 'ED4C6A63E0F8EB7FBD1D8B6A37513B3557DDA6F468A14737C7DE805BE71C657D2D',
 'TransactionType': 'Payment',
 'TxnSignature': 'B8A696B2000B1A147E70CD7D4185E26B9F34B9615BA5FDBDC27C8A9B0789F674B973569491BB82B0635072B814BAF598128641E77005D11CFC1D9E244191450D',
 'date': 743675772,
 'hash': '8628FE16875076B59364578E1EA5478FF1D173F095D0154F2F175524A4F1A8E8',
 'inLedger': 39806407,
 'ledger_index': 39806407,
 'meta': {'AffectedNodes': [{'ModifiedNode': {'FinalFields': {'Account': 'rGkJhixWWpXHPD4ebcrZAAr37bnMMguhj',
      'Balance': '9999998990',
      'Flags': 0,
      'OwnerCount': 0,
      'Sequence': 39806403},
     'LedgerEntryType': 'AccountRoot',
     'LedgerIndex': '668598A052707517F7EF0707978CD9A83BC7EFDC166576AF856BD67FCB8921F3',
     'PreviousFields': {'Balance': '100000

## **레저 정보 확인**

클라이언트에게 Ledger의 정보를 요청(request) 하면, 레저의 정보를 받아올 수 있습니다. 특히 계정의 정보를 가져올 때 유용합니다.

이를 위해서는 `Request` 객체를 생성하고, `client`의 `request` 메서드를 부르면 됩니다.

In [ ]:
from xrpl.clients import XRPLRequestFailureException
from xrpl.models.requests import (
    Request,
    AccountInfo,
    AccountTx,
)

In [ ]:
def request_ledger(client: JsonRpcClient, request: Request) -> dict:
    """
    XRPL 네트워크에 ledger 요청을 보냅니다.

    Args:
        client (JsonRpcClient): 요청을 보낼 클라이언트입니다.
        request (Request): 보낼 요청 객체입니다.

    Returns:
        dict: 요청의 결과를 포함하는 딕셔너리 객체입니다.

    Raises:
        XRPLRequestFailureException: 요청이 실패하면 발생합니다.
    """
    # Send request and get response
    response = client.request(request)

    # Raise exception if request failed
    if not response.is_successful():
        raise XRPLRequestFailureException(response.result)

    # Return result
    return response.result

### **계정 정보**

XRP 잔액이나, 계정의 설정(flag) 등을 확인할 수 있는 리퀘스트입니다.

In [ ]:
def get_account_info(
    client: JsonRpcClient, address: str, **kwargs
) -> dict:
    """
    XRPL 네트워크에서 이 계정의 정보를 가져옵니다.

    Args:
        client (JsonRpcClient): 요청을 보낼 클라이언트입니다.
        address (str): 계정 정보를 조회할 계정의 주소입니다.
        **kwargs: 추가적인 선택적 매개변수들입니다.

    Returns:
        dict: 이 계정의 정보를 포함하는 딕셔너리 객체입니다.
    """
    return request_ledger(client, AccountInfo(account=address, **kwargs))

In [ ]:
result = get_account_info(client=client, address=wallet.address)

In [ ]:
result

{'account_data': {'Account': 'rGkJhixWWpXHPD4ebcrZAAr37bnMMguhj',
  'Balance': '9999998990',
  'Flags': 0,
  'LedgerEntryType': 'AccountRoot',
  'OwnerCount': 0,
  'PreviousTxnID': '8628FE16875076B59364578E1EA5478FF1D173F095D0154F2F175524A4F1A8E8',
  'PreviousTxnLgrSeq': 39806407,
  'Sequence': 39806403,
  'index': '668598A052707517F7EF0707978CD9A83BC7EFDC166576AF856BD67FCB8921F3'},
 'account_flags': {'defaultRipple': False,
  'depositAuth': False,
  'disableMasterKey': False,
  'disallowIncomingCheck': False,
  'disallowIncomingNFTokenOffer': False,
  'disallowIncomingPayChan': False,
  'disallowIncomingTrustline': False,
  'disallowIncomingXRP': False,
  'globalFreeze': False,
  'noFreeze': False,
  'passwordSpent': False,
  'requireAuthorization': False,
  'requireDestinationTag': False},
 'ledger_current_index': 39806408,
 'validated': False}

### **계정의 트랜잭션 정보들 가져오기**

계정의 트랜잭션 정보들을 가져옵니다. `limit`은 검색할 거래의 개수로, 파라미터를 주지 않거나 0으로 주면 전부 가져옵니다.

In [ ]:
def get_account_transactions(
    client: JsonRpcClient, address: str, limit: int = 0, **kwargs
) -> dict:
    """
    XRPL 네트워크에서 이 계정의 거래 내역을 가져옵니다.

    Args:
        client (JsonRpcClient): 요청을 보낼 클라이언트입니다.
        address (Address): 거래 내역을 조회할 계정의 주소입니다.
        limit (Optional[int]): 검색할 거래의 최대 개수입니다. 0이면 모두 검색합니다. 기본값은 0입니다.
        **kwargs: 추가적인 선택적 매개변수들입니다.

    Returns:
        Result: 이 계정의 거래 내역을 포함하는 Result 객체입니다.
    """
    result = request_ledger(
        client, AccountTx(account=address, limit=limit, **kwargs)
    )
    return result["transactions"]

In [ ]:
result = get_account_transactions(client=client, address=wallet.address)

In [ ]:
result

[{'meta': {'AffectedNodes': [{'ModifiedNode': {'FinalFields': {'Account': 'rGkJhixWWpXHPD4ebcrZAAr37bnMMguhj',
       'Balance': '9999998990',
       'Flags': 0,
       'OwnerCount': 0,
       'Sequence': 39806403},
      'LedgerEntryType': 'AccountRoot',
      'LedgerIndex': '668598A052707517F7EF0707978CD9A83BC7EFDC166576AF856BD67FCB8921F3',
      'PreviousFields': {'Balance': '10000000000', 'Sequence': 39806402},
      'PreviousTxnID': 'E96CFD20ED955A905E33D0D376AC8AF74F095A13C6D09596B6D21248D360A018',
      'PreviousTxnLgrSeq': 39806402}},
    {'ModifiedNode': {'FinalFields': {'Account': 'rBALEkaidLzjynofASY8cfD85a5Vsp76ZQ',
       'Balance': '10000001000',
       'Flags': 0,
       'OwnerCount': 0,
       'Sequence': 39806404},
      'LedgerEntryType': 'AccountRoot',
      'LedgerIndex': 'E016AF54516F077BDCD20D13FF54D3BBD13C360C6489E59FF41C5F7864E40664',
      'PreviousFields': {'Balance': '10000000000'},
      'PreviousTxnID': '2BA612A06C74A4D3D805A74009853DC416CDFFA933A26996D1B41

## **토큰 (Token)**

`wonj`가 `CAT` 토큰의 발행자가 되어, `jay`에게로 `CAT` 토큰을 10개 전송하는 실습을 진행해보도록 하겠습니다.

In [ ]:
from xrpl.models.transactions import TrustSet
from xrpl.models.currencies import IssuedCurrency
from xrpl.models.requests import AccountLines

In [ ]:
wonj_wallet = generate_faucet_wallet(client=client, debug=True)
jay_wallet = generate_faucet_wallet(client=client, debug=True)

Attempting to fund address rBctcN3tBxDHmAM8WoXX5Lvz3TL8Uuq1tG
Faucet fund successful.
Attempting to fund address r3VkeEMt76v9iwrzdtW1L2DpeHZCyDn9JD
Faucet fund successful.


### **Trust Line**

XRP Ledger에서 토큰의 이동은 `trust line`을 통해서만 가능합니다.

또한 토큰을 받는 계정이 발행자의 계정으로 `trust line`을 설정해야 합니다.

`trust line`의 설정은 트랜잭션을 통해 설정이 가능합니다.

In [ ]:
def set_trust_line(
    client: JsonRpcClient,
    wallet: Wallet,
    token_symbol: str,
    issuer_address: str,
    limit: str | int,
    **kwargs,
) -> dict:
    """
    이 계정과 발행자 사이에 특정 토큰에 대한 trust line을 설정합니다.

    Args:
        client (JsonRpcClient): XRPL과 통신하기 위한 클라이언트 객체 입니다.
        wallet (Wallet): 트랜잭션을 제출하는 계정의 지갑 객체입니다.
        token_symbol (str): trust line의 토큰입니다.
        issuer_address (Address): 토큰의 발행자 주소입니다.
        limit (str | int): trust line의 한도입니다.
        **kwargs: 트랜잭션 설정에 추가로 전달할 선택적 매개변수입니다.

    Returns:
        dict: 트랜잭션의 결과입니다.
    """
    # create issued currency instance
    issued_currency = IssuedCurrency(currency=token_symbol, issuer=issuer_address)

    # convert to IssuedCurrencyAmount
    limit_amount = issued_currency.to_amount(value=str(limit))

    # create TrustSet transaction
    trust_set_tx = TrustSet(
        account=wallet.address,
        limit_amount=limit_amount,
        **kwargs,
    )

    # Submit transaction and return result
    return submit_transaction(
        client=client, wallet=wallet, transaction=trust_set_tx, check_fee=True
    )

`wonj`가 `CAT`의 발행자이므로, `jay`가 `wonj`에게 `CAT` 토큰에 대한 `trust line`을 설정합니다.

In [ ]:
result = set_trust_line(
    client=client,
    wallet=jay_wallet,
    token_symbol="CAT",
    issuer_address=wonj_wallet.address,
    limit="50",
)

In [ ]:
result

{'Account': 'r3VkeEMt76v9iwrzdtW1L2DpeHZCyDn9JD',
 'Fee': '10',
 'Flags': 0,
 'LastLedgerSequence': 39806432,
 'LimitAmount': {'currency': 'CAT',
  'issuer': 'rBctcN3tBxDHmAM8WoXX5Lvz3TL8Uuq1tG',
  'value': '50'},
 'Sequence': 39806411,
 'SigningPubKey': 'ED604F55824B727C2C044755403FC3E4D83FEC031035A92FD86302BC8E33EC2C04',
 'TransactionType': 'TrustSet',
 'TxnSignature': 'B8F1F79F6FBDAC7F081C2FC2314006CCE115A0DC444304C65807C4985ABC65C08FC5C45FD968DCC2D33FCC666D07EC9F467D31C3D9CF2CE3F5C739ABA145CA03',
 'date': 743675792,
 'hash': 'F90912BE53E6CDBBC012FAEA8E19F2F531F79EF69F69DECD7F8B4AF30A27A0EB',
 'inLedger': 39806413,
 'ledger_index': 39806413,
 'meta': {'AffectedNodes': [{'ModifiedNode': {'FinalFields': {'Account': 'r3VkeEMt76v9iwrzdtW1L2DpeHZCyDn9JD',
      'Balance': '9999999990',
      'Flags': 0,
      'OwnerCount': 1,
      'Sequence': 39806412},
     'LedgerEntryType': 'AccountRoot',
     'LedgerIndex': '6611529643B3A738E246DB7EFD5999B9747C094020E532C0302024E6AFA804E6',
     'Pr

이제 `jay`는 `wonj`로부터 발행된 `CAT` 토큰을 50개까지 보유할 수 있습니다.

In [ ]:
def send_token(
    client: JsonRpcClient,
    wallet: Wallet,
    destination_address: str,
    token_symbol: str,
    issuer_address: str,
    amount: str | int,
    **kwargs,
) -> dict:
    """
    계정에서 목적지 주소로 토큰을 보냅니다.

    Args:
        client (JsonRpcClient): XRPL과 통신하기 위한 클라이언트 객체 입니다.
        wallet (Wallet): 토큰을 보내는 계정의 지갑 객체입니다.
        destination_address (str): 토큰을 받을 계정의 주소입니다.
        token_symbol (str): 보낼 토큰의 이름입니다.
        issuer_address (str): 보낼 토큰의 발행자 주소입니다.
        amount (str | int): 보낼 토큰의 양입니다.
        **kwargs: 트랜잭션 설정에 추가로 전달할 선택적 매개변수입니다.

    Returns:
        dict: 트랜잭션의 결과입니다.
    """
    # create issued currency instance
    issued_currency = IssuedCurrency(currency=token_symbol, issuer=issuer_address)

    # convert to IssuedCurrencyAmount
    amount = issued_currency.to_amount(value=amount)

    # create payment transaction
    payment_tx = Payment(
        account=wallet.address,
        amount=amount,
        destination=destination_address,
        **kwargs,
    )

    # Submit transaction and return result
    return submit_transaction(
        client=client, wallet=wallet, transaction=payment_tx, check_fee=True
    )

이제 `wonj`가 `jay`에게 `CAT` 토큰을 10개 전송합니다.

In [ ]:
result = send_token(
    client=client,
    wallet=wonj_wallet,
    destination_address=jay_wallet.address,
    token_symbol="CAT",
    issuer_address=wonj_wallet.address,
    amount="10",
)

In [ ]:
result

{'Account': 'rBctcN3tBxDHmAM8WoXX5Lvz3TL8Uuq1tG',
 'Amount': {'currency': 'CAT',
  'issuer': 'rBctcN3tBxDHmAM8WoXX5Lvz3TL8Uuq1tG',
  'value': '10'},
 'Destination': 'r3VkeEMt76v9iwrzdtW1L2DpeHZCyDn9JD',
 'Fee': '10',
 'Flags': 0,
 'LastLedgerSequence': 39806449,
 'Sequence': 39806409,
 'SigningPubKey': 'EDD306D1C7E3A062CCCD743D5323FC47DEBD470980A24AC93F1AB3A79BF1C708EA',
 'TransactionType': 'Payment',
 'TxnSignature': 'D1BA283C9CBD3783BD1B601137EBBA6AB228A92725744DD2AACC814A332C8CE6AF5FE823EB146CF8097C3401265FC11E512967510FBAEFADA1AD15109DF7570B',
 'date': 743675851,
 'hash': '07D1534011C5D6F00A3BB41F41FFE778B7427B9034F9A368F9FE0593848C1555',
 'inLedger': 39806431,
 'ledger_index': 39806431,
 'meta': {'AffectedNodes': [{'ModifiedNode': {'FinalFields': {'Balance': {'currency': 'CAT',
       'issuer': 'rrrrrrrrrrrrrrrrrrrrBZbvji',
       'value': '10'},
      'Flags': 2162688,
      'HighLimit': {'currency': 'CAT',
       'issuer': 'rBctcN3tBxDHmAM8WoXX5Lvz3TL8Uuq1tG',
       'value': '0

### **계정의 Trust Line 확인**

Ledger로부터 정보를 요청해서 계정이 보유한 trust line들을 확인할 수 있습니다.

검색하길 원하는 토큰의 심볼을 매개변수로 주면, 그 토큰의 정보만 필터링하도록 하는 추가 코드가 있습니다.

In [ ]:
def get_trust_lines(
    client: JsonRpcClient,
    address: str,
    token_symbol: str = None,
    **kwargs
) -> dict:
    """
    이 계정의 trust line을 조회합니다.

    Args:
        client (JsonRpcClient): 요청을 보낼 클라이언트입니다.
        address (str): trust line을 조회할 계정의 주소입니다.
        token_symbol (Optional[str], optional): 조회할 토큰의 심볼입니다. None이면 모두 조회합니다. 기본값은 None입니다.
        **kwargs: 추가적인 선택적 매개변수들입니다.

    Returns:
        dict: 계정의 trust line을 포함하는 딕셔너리 객체입니다.
    """
    # Get trust lines
    result = request_ledger(client, AccountLines(account=address, **kwargs))

    # Filter by token_name (if provided)
    if token_symbol is not None:
        return [
            line for line in result["lines"] if line["currency"] == token_symbol
        ]

    # Return trust lines
    return result["lines"]

`wonj`와 `jay`의 `trust line`을 확인해보면 다음과 같습니다.

In [ ]:
wonj_lines = get_trust_lines(client=client, address=wonj_wallet.address, token_symbol="CAT")
jay_lines = get_trust_lines(client=client, address=jay_wallet.address, token_symbol="CAT")

In [ ]:
wonj_lines

[{'account': 'r3VkeEMt76v9iwrzdtW1L2DpeHZCyDn9JD',
  'balance': '-10',
  'currency': 'CAT',
  'limit': '0',
  'limit_peer': '50',
  'no_ripple': True,
  'no_ripple_peer': False,
  'quality_in': 0,
  'quality_out': 0}]

In [ ]:
jay_lines

[{'account': 'rBctcN3tBxDHmAM8WoXX5Lvz3TL8Uuq1tG',
  'balance': '10',
  'currency': 'CAT',
  'limit': '50',
  'limit_peer': '0',
  'no_ripple': False,
  'no_ripple_peer': True,
  'quality_in': 0,
  'quality_out': 0}]

## **끝으로 ..**

여기까지 이번 XRPL 개발자 밋업의 개발 세션까지 전부 끝이 났습니다!! 👏

모든 자료는 깃허브로 공개가 될 예정이고, 발표자료는 [Marp](https://marp.app)라는 툴을 이용해 마크다운으로 제작했습니다.

질문은 곧 있을 Q&A 세션이나 이메일(wonjae@catalyze-research.com) 또는 텔레그램(@wonj1012)으로  편하게 질문 주시면 답변해 드리겠습니다.

현재 카탈라이즈 리서치에서는 제가 XRP Ledger 개발 한국어 커리큘럼을 제작 중에 있습니다.

9월이나 10월에 공개가 될 예정인데, 많은 관심 부탁드리겠습니다!!

## Further Information

-   [XRPL Developer Portal](https://xrpl.org)
-   [XRPL Foundation](https://foundation.xrpl.org/)
-   [SHAMap](https://github.com/XRPLF/rippled/blob/develop/src/ripple/shamap/README.md)
-   [RPCA Whitepaper](https://ripple.com/files/ripple_consensus_whitepaper.pdf)
-   [Sologenic](https://www.sologenic.com/)
-   [XLS-30d AMM](https://opensource.ripple.com/docs/xls-30d-amm/amm-uc/)
-   [XLS-30d AMM (more technical)](https://github.com/XRPLF/XRPL-Standards/discussions/78)
-   [Hooks](https://xrpl-hooks.readme.io/)
-   [EVM Sidechain](https://opensource.ripple.com/docs/evm-sidechain/intro-to-evm-sidechain/)